In [1]:
import numpy as np
import pandas as pd
import xarray as xr
from virtualizarr import open_virtual_dataset
import glob
import os

In [2]:
# Test 2001-01 daily
path_glob = '../micasa-data/daily-0.1deg-final/holding/daily/2001/01/MiCASA_v1_flux*.nc4'

path_list = sorted(glob.glob(path_glob),key=os.path.getmtime)
path_list[0:3]

['../micasa-data/daily-0.1deg-final/holding/daily/2001/01/MiCASA_v1_flux_x3600_y1800_daily_20010101.nc4',
 '../micasa-data/daily-0.1deg-final/holding/daily/2001/01/MiCASA_v1_flux_x3600_y1800_daily_20010102.nc4',
 '../micasa-data/daily-0.1deg-final/holding/daily/2001/01/MiCASA_v1_flux_x3600_y1800_daily_20010103.nc4']

## Error debugging

In [3]:
path_test = path_list[0:3]

In [7]:
virtual_datasets_test = [open_virtual_dataset(filepath) for filepath in path_test]

# concat_test = xr.concat(virtual_datasets_test,dim='time')
concat_test = xr.concat(virtual_datasets_test,dim='time',coords='minimal',compat='override')
concat_test.virtualize.to_kerchunk('concat_test.json', format='json')

In [5]:
ds_concat = xr.open_dataset('concat_test.json',engine='kerchunk')
ds_concat

<xarray.Dataset> Size: 467MB
Dimensions:    (time: 3, lat: 1800, lon: 3600, nv: 2)
Coordinates:
  * lat        (lat) float64 14kB -89.95 -89.85 -89.75 ... 89.75 89.85 89.95
  * lon        (lon) float64 29kB -179.9 -179.8 -179.8 ... 179.8 179.8 179.9
  * nv         (nv) float64 16B nan 4.941e-324
  * time       (time) datetime64[ns] 24B 2001-01-01 NaT NaT
Data variables:
    ATMC       (time, lat, lon) float32 78MB ...
    FIRE       (time, lat, lon) float32 78MB ...
    FUEL       (time, lat, lon) float32 78MB ...
    NEE        (time, lat, lon) float32 78MB ...
    NPP        (time, lat, lon) float32 78MB ...
    Rh         (time, lat, lon) float32 78MB ...
    time_bnds  (time, nv) datetime64[ns] 48B ...

In [ ]:
# combine_test = xr.combine_nested(virtual_datasets_test,concat_dim=['time'])
combine_test = xr.combine_nested(virtual_datasets_test,concat_dim=['time'],coords='minimal',compat='override')
combine_test.virtualize.to_kerchunk('combine_test.json', format='json')

In [ ]:
ds_combine = xr.open_dataset('combine_test.json',engine='kerchunk')
ds_combine

In [10]:
# Loadable test
virtual_datasets = [open_virtual_dataset(filepath, loadable_variables=["time"]) for filepath in path_test]
virtual_ds_combine = xr.combine_nested(virtual_datasets, concat_dim=['time'], coords='minimal',compat='override')
virtual_ds_combine.virtualize.to_kerchunk('combined_test_loadable.json', format='json')

In [11]:
ds_loadable = xr.open_dataset('combined_test_loadable.json', engine='kerchunk', chunks={}) 
ds_loadable

<xarray.Dataset> Size: 467MB
Dimensions:    (time: 3, lat: 1800, lon: 3600, nv: 2)
Coordinates:
  * lat        (lat) float64 14kB -89.95 -89.85 -89.75 ... 89.75 89.85 89.95
  * lon        (lon) float64 29kB -179.9 -179.8 -179.8 ... 179.8 179.8 179.9
  * nv         (nv) float64 16B nan 4.941e-324
  * time       (time) datetime64[ns] 24B 2001-01-01 2001-01-02 2001-01-03
Data variables:
    ATMC       (time, lat, lon) float32 78MB dask.array<chunksize=(1, 600, 1200), meta=np.ndarray>
    FIRE       (time, lat, lon) float32 78MB dask.array<chunksize=(1, 600, 1200), meta=np.ndarray>
    FUEL       (time, lat, lon) float32 78MB dask.array<chunksize=(1, 600, 1200), meta=np.ndarray>
    NEE        (time, lat, lon) float32 78MB dask.array<chunksize=(1, 600, 1200), meta=np.ndarray>
    NPP        (time, lat, lon) float32 78MB dask.array<chunksize=(1, 600, 1200), meta=np.ndarray>
    Rh         (time, lat, lon) float32 78MB dask.array<chunksize=(1, 600, 1200), meta=np.ndarray>
    time_bnds  (time, nv) float64 48B dask.array<chunksize=(1, 2), meta=np.ndarray>

# Make virtualizarr store for MiCASA data

In [ ]:
# path_test = path_list

In [ ]:
# virtual_datasets = [open_virtual_dataset(filepath, loadable_variables=["time"], indexes={}) for filepath in path_test] #loadable item carries datetime correctly

In [ ]:
# virtual_ds_combine = xr.combine_nested(virtual_datasets, concat_dim=['time'], coords='minimal',compat='override')
# virtual_ds_combine

In [ ]:
# virtual_ds_combine.virtualize.to_kerchunk('combined.json', format='json')

## Open zarr dataset

In [ ]:
# one I just made
ds_combine = xr.open_dataset('combined.json', engine='kerchunk', chunks={}) 
ds_combine

In [ ]:
# old one all time
ds_old = xr.open_dataset('old/combined.json', engine='kerchunk', chunks={}) 
ds_old

## Concat vs combine nested?

In [ ]:
# # this Dataset wraps a bunch of virtual ManifestArray objects directly
# virtual_ds = xr.combine_nested(virtual_datasets, concat_dim=['time'])
# virtual_ds = xr.concat(virtual_datasets, dim='time')


# # cache the combined dataset pattern to disk, in this case using the existing kerchunk specification for re ference files
# virtual_ds.virtualize.to_kerchunk('combined_test.json', format='json')
# # virtual_ds.virtualize.to_kerchunk('combined_test.json', format='parquet')